## Introduction to Estimation

Estimation is a branch of the Analysis Phase to use when you are testing for a continuous variable.  

In [ ]:
# Import libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Workshop Functions
import sys
sys.path.append('..')
from Wksp722_functions import * 

In [ ]:
# Read in data
housing = pd.read_csv("housing.csv")
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.isnull().sum()

### Next Steps
Let's try to replace the missing 'total_bedrooms' column with the median value for that variable.  

In [ ]:
median = housing.loc[:,"total_bedrooms"].median()
housing.loc[:,"total_bedrooms"].fillna(median, inplace=True)

housing.isnull().sum()

Next, let's look at the variables and see how they're correlated

In [ ]:
corr_matrix=housing.corr(numeric_only=True)
corr_matrix.median_house_value.sort_values(ascending=False)

It looks like 'median_income' has a high correlation to the median_house_value.  

Another Kaggle user created a new variable by dividing the total_rooms and total_bedrooms by households.  On their own, total_rooms, total_bedrooms and households are good indicators of housing density in an area.  The new normalized values give the number of rooms and bedrooms per household, which can be a better indicator of house side.  

In [ ]:
housing.loc[:,'rooms_per_household'] = housing.loc[:,'total_rooms']/housing.loc[:,'households']
housing.loc[:,'bedrooms_per_household'] = housing.loc[:,'total_bedrooms']/housing.loc[:,'households']

In [ ]:
# re-run correlation
corr_matrix=housing.corr(numeric_only=True)
corr_matrix.median_house_value.sort_values(ascending=False)

The 'rooms_per_household' had a high correlation to the median_house_value.  This *may* be useful to analyses models.  

Lattitude also has a high correlation value.  I'm using code from a Kaggle notebook found here: https://www.kaggle.com/code/mostafaashraf1/california-housing-prices

The map shows clustering of high value homes around the coast between San Fransciso and San Diego.  These are also highly populated areas. 

In [ ]:
M3L4_CA_plot(housing)

In [ ]:
#### Convert ocean_proximity to numerical columns using One-Hot encoding
dfTemp = pd.get_dummies(housing.loc[:,'ocean_proximity'])

housing = pd.concat([housing,dfTemp], axis=1)
housing.drop('ocean_proximity', axis=1,inplace=True)
housing.head(2)

### Normalizing variables
Some estimation algorithms require normally distributed variables.  We can use the RobustScaler function in Scikit-learn to normalize the variables.  But first, let's take a look at the distributions:

In [ ]:
housing.hist(bins=50,figsize=(20,10))
plt.show()

We see that the distributions that seem to follow a Gaussian distribution (total_rooms, total_bedrooms, population, households, median_income, median_house_value), but have a longer right side tail.  RobustScaler will remove the median and scale the variance.   

In [ ]:
x = housing.drop(columns="median_house_value")
y = housing.loc[:,'median_house_value']

from sklearn.preprocessing import RobustScaler
ro_scaler = RobustScaler()
housingScaled = ro_scaler.fit_transform(x) #<-- only scale the input variables, not the target

housingScaled = pd.DataFrame(housingScaled, columns=x.columns)
housingScaled.hist(bins=50,figsize=(20,10));

While the distribution shapes for each variable have remained the same, the ranges have been adjusted to account for the longer tails and each variable has been normalized.  

We are now ready to work on the estimation algorithms:
* Linear Regression
* Decision Tree
* Random Forest (Regression)

In [ ]:
# First let's define training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=1)

#### Linear Regression

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression
LR = LinearRegression().fit(x_train,y_train)
y_pred_LR = LR.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score
rmse_LR = np.sqrt(mean_squared_error(y_pred_LR, y_test))
R2_LR = r2_score(y_pred_LR, y_test)

In [ ]:
# Let's plot the actual vs predicted values
M3L4_Predicted_Plot(x_test, y_test, y_pred_LR, numPts = 500)

#### Decision Tree
First let's start with a small decision tree so we can take a look at it

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeRegressor
Tree = DecisionTreeRegressor(random_state=1, max_depth=3).fit(x_train, y_train)
y_pred_Tree = Tree.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score
print('RMSE = ',np.sqrt(mean_squared_error(y_pred_Tree, y_test)))
print('R2 = ', r2_score(y_pred_Tree, y_test))

In [ ]:
from sklearn import tree
fig = plt.figure(figsize=(25,20))
tree.plot_tree(Tree,feature_names=x_test.columns)

fig.savefig('treeRegression.pdf')

In [ ]:
# Decision Tree - Full Length
from sklearn.tree import DecisionTreeRegressor
Tree = DecisionTreeRegressor(random_state=1, max_depth=None).fit(x_train, y_train)
y_pred_Tree = Tree.predict(x_test)

rmse_Tree = np.sqrt(mean_squared_error(y_pred_Tree, y_test))
R2_Tree = r2_score(y_pred_Tree, y_test)

print('mean house price = ', y_test.mean())
print(rmse_Tree, R2_Tree)
print('max tree depth = ', Tree.tree_.max_depth)

In [ ]:
# Random Forest 
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(n_estimators = 100, max_depth=None, random_state=1)
RF.fit(x_train, y_train)
y_pred_RF = RF.predict(x_test)

from sklearn.metrics import mean_squared_error, r2_score
rmse_RF = np.sqrt(mean_squared_error(y_pred_RF, y_test))
R2_RF = r2_score(y_pred_RF, y_test)

#### Performance Summary

In [ ]:
data = {'Estimator':['Linear Regression', 'Decision Tree','Random Forest'],
       'RMSE':[rmse_LR, rmse_Tree,rmse_RF],
       'R2 Score':[R2_LR, R2_Tree, R2_RF]}

EstimationMetrics = pd.DataFrame(data)
#print(EstimationMetrics)
display(EstimationMetrics) #<--- Nicer alternative to print for dataframes